In [89]:
import cv2
import numpy as np
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import os
import scipy
from skimage import io, transform
from math import sin
from math import cos
from math import radians
import pandas as pd



images_path   = "/home/ibrahim/Projects/Datasets/Ch2_001/center/"
dataset_path  = "/home/ibrahim/Projects/Datasets/Ch2_001/"
projects_path = "/home/ibrahim/Projects/Self_Maneuvering_UGV/"

os.chdir(dataset_path)
os.getcwd()


'/home/ibrahim/Projects/Datasets/Ch2_001'

In [90]:
# Car Dataset Creator Class:
class carCam(Dataset):
    def __init__(self, csv_file, root_dir, transform = None):
        self.csv_file = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_name = os.path.join(self.root_dir + 'center',
                                  str(self.csv_file.iloc[idx,0]) + '.jpg')
        
        image = cv2.imread(image_name)
        steering_angle = self.csv_file.iloc[idx,1]

        if self.transform:
            image = self.transform(image)   
        sample = {"image": image, "steering_angle": steering_angle}

        return sample


# Transform Dataset Class:
class CropTransform(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        pass

    def __call__(self, image):
        return (image[self.x:, self.y:])

In [92]:
crop = CropTransform(150, 0)

car_cam = carCam(csv_file = dataset_path + 'final_example.csv',
                 root_dir = dataset_path, transform = Crop(150,0))

for i in range(500):
    sample = car_cam[i]
    # print(sample["steering_angle"])
    cv2.imshow("frame",sample["image"])
    cv2.waitKey(1)
    

cv2.destroyAllWindows()
car_cam.csv_file.head(3)

,frame_id,steering_angle
0,1479425441182877835,-0.373665
1,1479425441232704425,-0.065396
2,1479425441282730750,-0.160735


In [94]:
dataloader = DataLoader(car_cam, batch_size=4,
                        shuffle=False, num_workers=4)

In [101]:
def showCarData(sample_batched):
    images_b, steering_b = \
        sample_batched['image'], sample_batched['steering_angle']

    bs = len(images_b)

    for i in range(bs):
        cv2.imshow("frame", images_b[i].numpy())
        cv2.waitKey(1)

count = 0
for i, sample_batched in enumerate(dataloader):
    showCarData(sample_batched)
    count += i

print(count)
cv2.destroyAllWindows()

984906
